# Instrumenting a LlamaCloud Agent (with Arize Phoenix) 

<a href="https://colab.research.google.com/github/run-llama/llamacloud-demo/blob/main/examples/tracing/llamacloud_tracing_phoenix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook we show you how to use our native integration with LlamaTrace (Arize Phoenix) in order to log full traces from an agent built over LlamaCloud Retrievers.

LlamaTrace is a collaboration between Arize and LlamaIndex to deliver hosted observability and evals with native support for LlamaIndex.

## Setup

Install core packages, download files, parse documents.

In [ ]:
!pip install llama-index
!pip install llama-index-core
!pip install llama-index-embeddings-openai
!pip install llama-index-question-gen-openai
!pip install llama-index-postprocessor-flag-embedding-reranker
!pip install git+https://github.com/FlagOpen/FlagEmbedding.git
!pip install llama-parse

In [ ]:
# download Apple 
!wget "https://s2.q4cdn.com/470004039/files/doc_earnings/2023/q4/filing/_10-K-Q4-2023-As-Filed.pdf" -O data/apple_2023.pdf
!wget "https://s2.q4cdn.com/470004039/files/doc_financials/2022/q4/_10-K-2022-(As-Filed).pdf" -O data/apple_2022.pdf
!wget "https://s2.q4cdn.com/470004039/files/doc_financials/2021/q4/_10-K-2021-(As-Filed).pdf" -O data/apple_2021.pdf
!wget "https://s2.q4cdn.com/470004039/files/doc_financials/2020/ar/_10-K-2020-(As-Filed).pdf" -O data/apple_2020.pdf
!wget "https://www.dropbox.com/scl/fi/i6vk884ggtq382mu3whfz/apple_2019_10k.pdf?rlkey=eudxh3muxh7kop43ov4bgaj5i&dl=1" -O data/apple_2019.pdf

# download Tesla
!wget "https://ir.tesla.com/_flysystem/s3/sec/000162828024002390/tsla-20231231-gen.pdf" -O data/tesla_2023.pdf
!wget "https://ir.tesla.com/_flysystem/s3/sec/000095017023001409/tsla-20221231-gen.pdf" -O data/tesla_2022.pdf
!wget "https://www.dropbox.com/scl/fi/ptk83fmye7lqr7pz9r6dm/tesla_2021_10k.pdf?rlkey=24kxixeajbw9nru1sd6tg3bye&dl=1" -O data/tesla_2021.pdf
!wget "https://ir.tesla.com/_flysystem/s3/sec/000156459021004599/tsla-10k_20201231-gen.pdf" -O data/tesla_2020.pdf
!wget "https://ir.tesla.com/_flysystem/s3/sec/000156459020004475/tsla-10k_20191231-gen_0.pdf" -O data/tesla_2019.pdf

Some OpenAI and LlamaParse details. The OpenAI LLM is used for response synthesis.

In [15]:
# llama-parse is async-first, running the async code in a notebook requires the use of nest_asyncio
import nest_asyncio
nest_asyncio.apply()

In [2]:
import os
# API access to llama-cloud
os.environ["LLAMA_CLOUD_API_KEY"] = "llx-"

In [3]:
# Using OpenAI API for embeddings/llms
os.environ["OPENAI_API_KEY"] = "sk-"

In [16]:
from llama_index.llms.openai import OpenAI
from llama_index.core import VectorStoreIndex, Settings

llm = OpenAI(model="gpt-4o", temperature=0)

Settings.llm = llm

### Setup Observability

We setup the LlamaTrace integration below.

If you haven't already done so, make sure to create an account here: https://llamatrace.com/login. Then create an API key and put it in the `PHOENIX_API_KEY` variable below.

In [ ]:
!pip install -U llama-index-callbacks-arize-phoenix

In [3]:
# setup Arize Phoenix for logging/observability
import llama_index.core
import os

PHOENIX_API_KEY = "<PHOENIX_API_KEY>"
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"api_key={PHOENIX_API_KEY}"
llama_index.core.set_global_handler(
    "arize_phoenix",
    endpoint="https://llamatrace.com/v1/traces"
)

## Load Documents into LlamaCloud

The first order of business is to download the 5 Apple and Tesla 10Ks and upload them into LlamaCloud.

You can easily do this by creating a pipeline and uploading docs via the "Files" mode.

After this is done, **note down your index name and project name** and proceed to the next section.

## Define LlamaCloud Retriever over Documents

In this section we define a LlamaCloud Retriever over these SEC 10K Filings. 

LlamaCloud natively uses LlamaParse + integrated table parsing + indexing to give back a retrieval endpoint over these documents.

We primarily use the default settings, without modification.

In [4]:
from llama_index.indices.managed.llama_cloud import LlamaCloudIndex

index = LlamaCloudIndex(
  name="apple_tesla_demo_base",  ## TODO: put index name here
  project_name="llamacloud_demo",  ## TODO: put project name here
  # api_key="llx-"
)

In [5]:
query_engine = index.as_query_engine(rerank_top_n=5, dense_similarity_top_k=15)

In [6]:
response = query_engine.query("What are risk factors for tesla in 2022")

In [7]:
print(str(response))

Tesla faced several risk factors in 2022, including:

1. **Competition**: Tesla faced competition in both its automotive and energy generation and storage businesses from established and new manufacturers, developers, installers, and service providers. Decreases in electricity prices from utilities or other renewable sources could make Tesla's products less attractive.

2. **Operational Issues**: Potential issues with lithium-ion cells or other components manufactured at Tesla's Gigafactories could harm production and profitability. Bottlenecks in manufacturing lines and the need to ramp up production capacity were also concerns.

3. **International Operations**: Tesla faced risks associated with maintaining and expanding international operations, including regulatory, political, economic, tax, and labor conditions in various jurisdictions.

4. **Product Performance**: Defects or performance issues in Tesla's products, including software-related problems, could harm the company's reput

## Define Custom Sub-Question Query Engine

Define a custom query engine with tool retrieval enabled.

In [8]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

In [9]:
query_engine_tool = QueryEngineTool(
    query_engine=query_engine,
    metadata=ToolMetadata(
        name=f"query_tool",
        description=(
            f"Search tool for Apple or Tesla financials for any year (please specify)"
        )
    )
)

In [10]:
from llama_index.core.agent import FunctionCallingAgentWorker

agent = FunctionCallingAgentWorker.from_tools(
    [query_engine_tool], llm=llm, verbose=True
).as_agent()

In [11]:
response = agent.query(
    "Get Property, plant and equipment assets for Tesla and Apple in 2022"
)

Added user message to memory: Get Property, plant and equipment assets for Tesla and Apple in 2022
=== Calling Function ===
Calling function: query_tool with args: {"input": "Tesla Property, plant and equipment assets 2022"}
=== Function Output ===
In 2022, Tesla's property, plant, and equipment assets included ongoing construction projects such as Gigafactory Texas and Gigafactory Berlin-Brandenburg, as well as equipment and tooling related to manufacturing. Completed assets were transferred to their respective asset classes and depreciation began when the assets were ready for use. Depreciation expense for the year was $2.42 billion. Additionally, Tesla capitalized gross costs of $2.01 billion related to production equipment under its arrangement with Panasonic.
=== Calling Function ===
Calling function: query_tool with args: {"input": "Apple Property, plant and equipment assets 2022"}
=== Function Output ===
In 2022, Apple's property, plant, and equipment assets were valued at $42,1

In [12]:
print(response.source_nodes[3].get_content(metadata_mode="all"))

file_size: 1650825
last_modified_at: 2024-07-10T00:58:22
file_path: tesla_2022.pdf
file_name: tesla_2022.pdf
pipeline_id: c62378b1-92ab-43ca-9e3c-b1fca4c5f73f

Leasehold improvements are depreciated on a straight-line basis over the shorter of their estimated useful lives or the terms of the related leases.

Upon the retirement or sale of our property, **plant** and **equipment**, the cost and associated accumulated depreciation are removed from the consolidated balance sheet, and the resulting gain or loss is reflected on the consolidated statement of operations. Maintenance and repair expenditures are expensed as incurred while major improvements that increase the functionality, output or expected life of an asset are capitalized and depreciated ratably over the identified useful life.

Interest expense on outstanding debt is capitalized during the period of significant capital asset construction. Capitalized interest on construction in progress is included within Property, **plant**

In [13]:
# print(str(response))
print(response.source_nodes[1].get_content(metadata_mode="all"))

file_size: 984581
last_modified_at: 2024-07-10T00:58:22
file_path: tesla_2023.pdf
file_name: tesla_2023.pdf
pipeline_id: c62378b1-92ab-43ca-9e3c-b1fca4c5f73f

Construction in progress is primarily comprised of ongoing construction and expansion of our facilities, and **equipment** and tooling related to the manufacturing of our products. Completed **assets** are transferred to their respective asset classes and depreciation begins when an asset is ready for its intended use.

Depreciation expense during the years ended December 31, 2023, **2022** and 2021 was $3.33 billion, $2.42 billion and $1.91 billion, respectively.

Panasonic has partnered with us on Gigafactory Nevada with investments in the production **equipment** that it uses to manufacture and supply us with battery cells. Under our arrangement with Panasonic, we plan to purchase the full output from their production **equipment** at negotiated prices. As the terms of the arrangement convey a finance lease under ASC 842, we a

In [14]:
print(str(response))

In 2022, the property, plant, and equipment assets for Tesla and Apple were as follows:

- **Tesla**: Tesla's property, plant, and equipment assets included ongoing construction projects such as Gigafactory Texas and Gigafactory Berlin-Brandenburg, as well as equipment and tooling related to manufacturing. Completed assets were transferred to their respective asset classes, and depreciation began when the assets were ready for use. Depreciation expense for the year was $2.42 billion. Additionally, Tesla capitalized gross costs of $2.01 billion related to production equipment under its arrangement with Panasonic.

- **Apple**: Apple's property, plant, and equipment assets were valued at $42,117 million.


In [33]:
response = agent.query("Tell me risk factors for both Tesla and Apple in 2023")

Added user message to memory: Tell me risk factors for both Tesla and Apple in 2023
=== Calling Function ===
Calling function: query_tool with args: {"input": "Tesla risk factors 2023"}
=== Function Output ===
Tesla's risk factors in 2023 include potential increases in costs and management challenges, defects or delays in product performance, product liability claims, maintaining public credibility, delays in production and scaling, supplier issues, financing program risks, union activities, product recalls, warranty reserves, and cybersecurity threats. These risks could impact Tesla's business, financial condition, and operating results.
=== Calling Function ===
Calling function: query_tool with args: {"input": "Apple risk factors 2023"}
=== Function Output ===
Apple's risk factors in 2023 include a variety of financial, macroeconomic, industry, legal, and general risks. Financial risks involve fluctuations in quarterly net sales and profit margins, exposure to foreign exchange rate m

In [34]:
print(str(response))

### Tesla's Risk Factors in 2023

1. **Cost Increases and Management Challenges**: Potential increases in costs and challenges in managing them.
2. **Product Performance Issues**: Defects or delays in product performance.
3. **Product Liability Claims**: Risks associated with claims related to product liability.
4. **Public Credibility**: Maintaining public credibility is crucial.
5. **Production and Scaling Delays**: Delays in production and scaling operations.
6. **Supplier Issues**: Problems with suppliers could impact operations.
7. **Financing Program Risks**: Risks related to financing programs.
8. **Union Activities**: Potential union activities could affect operations.
9. **Product Recalls**: Risks associated with product recalls.
10. **Warranty Reserves**: Managing warranty reserves effectively.
11. **Cybersecurity Threats**: Threats to cybersecurity could impact operations.

### Apple's Risk Factors in 2023

1. **Financial Risks**:
   - Fluctuations in quarterly net sales and